# Insert here your bidding quantity list :

In [1]:
import pandas as pd
from pandas import ExcelFile
import numpy as np

# In my example the bidding quantity are taken from the Wind Data :
WindData = pd.read_excel('WindForecast_DA_12.xlsx')

# Import market price data for 2017 from Nordpool :
SpotData = pd.read_excel('DK1_2017.xlsx')
df2016 = pd.read_excel('DK1_2016.xlsx')   

In [2]:
#calculate the average pi+ and pi- for every hour with 2016 data.
df2016_hours=df2016[["Hours","Pi+","Pi-"]].groupby("Hours",as_index=False).agg("sum")
df2016_hours["Phi+"]=df2016_hours["Pi+"]/366
df2016_hours["Phi-"]=df2016_hours["Pi-"]/366

#then calculate the ratio r for every hour
df2016_hours["r"]=df2016_hours["Phi+"]/(df2016_hours["Phi+"]+df2016_hours["Phi-"])

#print Dataframe
df2016_hours.head(24)

,Hours,Pi+,Pi-,Phi+,Phi-,r
0,00 - 01,770.68,567.55,2.105683,1.550683,0.575895
1,01 - 02,751.04,687.88,2.052022,1.879454,0.521947
2,02 - 03,840.78,708.48,2.297213,1.935738,0.542698
3,03 - 04,804.71,651.59,2.198661,1.780301,0.552572
4,04 - 05,979.54,535.35,2.676339,1.462705,0.646608
5,05 - 06,1103.20,358.12,3.014208,0.978470,0.754934
6,06 - 07,1221.54,367.56,3.337541,1.004262,0.768699
7,07 - 08,1139.04,876.91,3.112131,2.395929,0.565014
8,08 - 09,1386.10,1297.28,3.787158,3.544481,0.516550
9,09 - 10,1501.49,999.68,4.102432,2.731366,0.600315


In [3]:
#function F-1 of the power generation distribution
def inv(i,r):
    #we find in which interval of quantile r is
    b=100*r//5  #eucledian equation 
    qp=int(5*(b+1))
    qm=int(5*b)
    
    #we approximate the inverse function with a linear function between this two quantiles
    res=((WindData["q"+str(qp)][i]-WindData["q"+str(qm)][i])/5) *(100*r-qm) + WindData["q"+str(qm)][i]
    return(res)

In [4]:
#Size of 2017 data
L=WindData.shape[0]

#Bid array
BID=[0]*L

#On the first day, we bid with r onlt trained with 2016 data. Other bid will be calculated after.
Phiplus=list(df2016_hours["Phi+"])+[0]*(L-24)
Phiminus=list(df2016_hours["Phi-"])+[0]*(L-24)
R=list(df2016_hours["r"])+[0]*(L-24)

#Once that we have a new 2017 value, we update the average phi+ and phi-
for i in range(24,L):
    t=366+i//24-1 #number of value used to calculate the average phi+ and phi - on day i
    hour=i%7 #hour of the day
    Phiplus[i]=Phiplus[i-24]*(t/(t+1)) + (SpotData['Clearing'][i-24]-SpotData['Down'][i-24])/(t+1)
    Phiminus[i]=Phiminus[i-24]*(t/(t+1)) + (SpotData['Up'][i-24]-SpotData['Clearing'][i-24])/(t+1)
    R[i]=Phiplus[i]/(Phiplus[i]+Phiminus[i])

#Finds the bid with the right r and the inverse function of the distribution
for i in range(L):
    BID[i]=inv(i,R[i])

In [5]:
#our final bid array
BID

[143599.50004109906,
 141966.26077891752,
 142953.25304984316,
 140989.83732747374,
 145295.2773204655,
 148253.43456600883,
 142498.72928072495,
 133976.920732161,
 71690.37103205659,
 79029.0485452808,
 64493.44702578503,
 66017.94105114433,
 69577.45471590446,
 98235.1184839155,
 89977.98439284353,
 91528.56894317461,
 101072.58658508495,
 111339.21445011364,
 112104.14433149285,
 105407.87459241535,
 99526.34324664506,
 101223.85542858312,
 95680.21211249371,
 120894.70353982301,
 67901.06697425211,
 69476.50120922754,
 80650.65318921975,
 70079.77278033372,
 96170.34970195856,
 91498.68135658173,
 84095.8400981688,
 38400.87162380019,
 31087.54312844249,
 46334.05676800102,
 82236.98118966972,
 107848.50044778793,
 126989.77970131751,
 138156.61768765264,
 122257.28114596775,
 112125.19388343912,
 137730.19910937344,
 138867.83406741967,
 120582.13757159811,
 117541.65873673029,
 110272.00535695827,
 114008.16476889425,
 102810.74753650292,
 101178.00442477876,
 132713.0645929454,

# Calculation code :

### Data :

#### Import Data :

In [6]:
# Import data for measurements and forecast made at 12:00 Day-1 :
WindData = pd.read_excel('WindForecast_DA_12.xlsx')

# Import market price data for 2017 from Nordpool :
SpotData = pd.read_excel('DK1_2017.xlsx')

# Check : Print the name of the excel file columns
print(WindData.columns)
print(SpotData.columns)

Index(['day', 'hour', 'dato', 'dati', 'ForeTime', 'hors', 'meas', 'fore', 'q5',
       'q10', 'q15', 'q20', 'q25', 'q30', 'q35', 'q40', 'q45', 'q50', 'q55',
       'q60', 'q65', 'q70', 'q75', 'q80', 'q85', 'q90', 'q95', 'Qr'],
      dtype='object')
Index(['Date', 'Hours', 'Clearing', 'Up', 'Down', 'Pi+', 'Pi-'], dtype='object')


#### Extract meaningful parts :

In [7]:
# Wind power measure, in kW :
Measure = WindData['meas']

# Market clearing price, in €/MWh :
LambdaS = SpotData['Clearing']

# Extract market regulation price, in €/MWh :
LambdaUP = SpotData['Up']
LambdaDOWN = SpotData['Down']

# Number of time units :
L = len(Measure)

#### Market properties :

We build an indicator dP which will give the sign for the type of regulation needed for the market:

#dP = -1 for down-regulation

#dP = 1 for up-regulation

#dP = 0 when the market is perfectly balanced

In [8]:
dP = [0]*L
for k in range (0, L, 1) :
    lambda_up = LambdaUP[k]
    lambda_down = LambdaDOWN[k]
    lambdaS = LambdaS[k]
    
    up = lambda_up - lambdaS      # lambda_up >= lambdaS
    down = lambdaS - lambda_down  # lambda_down <= lambdaS
    
    if lambda_up == lambda_down:
        dP[k] = 0
    elif up > down :
        dP[k] = -1                # Need of upward regulation, negative imbalance
    else :
        dP[k] = 1                 # Need of downward regulation, positiv imbalance


Now we can determine the balancing price LambdaB according to the market imbalance :

In [9]:
LambdaB = [0]*L
for k in range (0, L, 1):
    dp = dP[k]
    if dp < 0 :
        LambdaB[k] = LambdaUP[k]     # Upward regulation
    elif dp > 0 :
        LambdaB[k] = LambdaDOWN[k]   # Downward regulation
    else :
        LambdaB[k] = LambdaS[k]      # Balanced Market


Adapt bid and real values when the producer is not scheduled :

In [10]:
RealBID = [0]*L
RealMEAS = [0]*L

for k in range (0,L,1):
    if LambdaS[k] < 0:
        RealBID[k] = 0
        RealMEAS[k] = 0
    else :
        RealBID[k] = BID[k]
        RealMEAS[k]= Measure[k]

#### Individual imbalance :

An indicator of the producer imbalance is defined similarly to the Market imbalance indicator dP :

In [11]:
SURPLUS = [0]*L      # Production surplus at time t, in kW
LACK = [0]*L         # Lack of production at time t, in kW
IMBAL = [0]*L        # Imbalance indicator

for k in range (0, L, 1) :
    real = RealMEAS[k]
    bid = RealBID[k]
    
    if real > bid :
        IMBAL[k] = 1              # The producer produces more than forecasted, his imbalance is positiv.
        SURPLUS[k] = real-bid
        LACK[k] = 0
    elif real < bid :
        IMBAL[k] = -1             # The producer doesn't produce enough, his imbalance is negativ.
        SURPLUS[k] = 0
        LACK[k] = real-bid       
    else :
        IMBAL[k] = 0              # The production bid was exact.
        SURPLUS[k] = 0
        LACK[k] = 0

#### Day Ahead Revenue :

In [12]:
RevDA = [0]*L

for k in range (0, L, 1):
    
    if LambdaS[k] >= 0 :
        lambdaS = LambdaS[k]          # 0.001 factor due to conversion from MWh to kWh
        bid = RealBID[k]
        RevDA[k] = lambdaS*bid*0.001

#### Balancing Market Revenue :

In [13]:
RevB = [0]*L              # Revenue of balancing market
LambdaREG = [0]*L         # Regulation price applied, LambdaS or LambdaB

for k in range (0, L, 1):
    real = RealMEAS[k]
    bid = RealBID[k]
    market_imbal = dP[k]
    indiv_imbal = IMBAL[k]
    
    lambdaB = LambdaB[k]
    lambdaS = LambdaS[k]
    
    # Case of upward regulation, the producer helps if he produces more than forecasted ie. his imbalance is positiv
    if market_imbal < 0 :
        if indiv_imbal >= 0 :
            revB = lambdaS*(real-bid)*0.001
            lambdaREG = lambdaS
        else :
            revB = lambdaB*(real-bid)*0.001
            lambdaREG = lambdaB
    # Case of downward regualtion, the producer helps if he produces less than forecasted ie. his imbalance is negativ
    elif market_imbal > 0 :
        if indiv_imbal <= 0 :
            revB = lambdaS*(real-bid)*0.001
            lambdaREG = lambdaS
        else :
            revB = lambdaB*(real-bid)*0.001
            lambdaREG = lambdaB
    # Case of balanced market, the producer's imbalance is not penalized :
    else :
        revB = lambdaS*(real-bid)*0.001
        lambdaREG = lambdaS
    
    RevB[k] = revB
    LambdaREG[k] = lambdaREG

# Results

In [14]:
# Net revenue :
RevNET = [0]*L
for k in range (0,L, 1):
    RevNET[k] = RevDA[k] + RevB[k]

RevNET_annual = sum(RevNET)
print('The net annual revenue is, in €:')
print(RevNET_annual)
print('Composed of a day-ahead revenue, in €:')
print(sum(RevDA))
print('and a balancing revenue, in €:')
print(sum(RevB))

The net annual revenue is, in €:
19152026.568970878
Composed of a day-ahead revenue, in €:
21920113.8534662
and a balancing revenue, in €:
-2768087.2844953476


In [15]:
# Real generation VS scheduled generation :
RealGene = sum(RealMEAS)*0.001
SchedGene = sum(RealBID)*0.001
print('The annual real generation is, in MWh:')
print(RealGene)
print('Against a annual scheduled generation of, in MWh:')
print(SchedGene)

The annual real generation is, in MWh:
708448.295
Against a annual scheduled generation of, in MWh:
776554.3205221432


In [16]:
# Surplus VS Shortage :
TotSurplus = sum(SURPLUS)*0.001
TotShortage = sum(LACK)*0.001
print('The global generation surplus in 2017 is, in MWh:')
print(TotSurplus)
print('& the global generation shortage in 2017 is, in MWh:')
print(TotShortage)
print('Representing a share of the global production, in %:')
print(100*(TotSurplus-TotShortage)/RealGene)

The global generation surplus in 2017 is, in MWh:
55099.66993627395
& the global generation shortage in 2017 is, in MWh:
-123205.69545841837
Representing a share of the global production, in %:
25.168437365593814


In [17]:
# Imbalance cost :

# Cost of upward regulation, when the producer is in shortage :
UPCOST = [0]*L
# Revenue when downward regulation, when the producer is in surplus :
DOWNCOST = [0]*L

# Opportunity loss cost : when the producer generates less than schedule and have to buy electricity more expensive than its own : :
OPCOST_SHORTAGE = [0]*L
# Opportunity loss cost : when the producer generates more than schedule and sell it for <= price than lambdaS :
OPCOST_SURPLUS = [0]*L

for k in range (0,L,1):
    UPCOST[k]=0.001*LACK[k]*LambdaREG[k]
    DOWNCOST[k]=0.001*SURPLUS[k]*LambdaREG[k]
    
    OPCOST_SHORTAGE[k]=0.001*LACK[k]*(LambdaREG[k]-LambdaS[k])
    OPCOST_SURPLUS[k]=0.001*SURPLUS[k]*(LambdaREG[k]-LambdaS[k])
    
print('The global shortage over the year create a opportunity loss of, in €:')
print(sum(OPCOST_SHORTAGE))
print('The global surplus over the year create a opportunity loss of, in €:')
print(sum(OPCOST_SURPLUS))
print('Giving a global opportunity loss of, in € :')
print(sum(OPCOST_SHORTAGE)+sum(OPCOST_SURPLUS))

# Just to check : The global opportunity loss should be equal to 
# the difference between the net revenue of this strategy and the ideal revenue.

The global shortage over the year create a opportunity loss of, in €:
-341391.8155603335
The global surplus over the year create a opportunity loss of, in €:
-221992.99037882415
Giving a global opportunity loss of, in € :
-563384.8059391576


In [18]:
# Average regulation prices per unit :
  
    # Upward and downward regulation, in €/MWh :
avUP = sum(UPCOST)/(0.001*sum(LACK))
avDOWN = sum(DOWNCOST)/(0.001*sum(SURPLUS))


print('The average upward regulation price is, in €/MWh:')
print(avUP)
print('The average downward regulation price is, in €/MWh:')
print(avDOWN)

print('The average energy price, in €/MWh:')
print(sum(RevNET)/RealGene)

The average upward regulation price is, in €/MWh:
34.37325453574145
The average downward regulation price is, in €/MWh:
26.62254506145801
The average energy price, in €/MWh:
27.033767607515912


In [19]:
# Performance ratio
Perf = sum(RevNET)/19715411.37491005
print('The performance ratio of this strategy is:')
print(Perf)

The performance ratio of this strategy is:
0.9714241414888183


# Results analysis

#### Import useful packages

In [20]:
from matplotlib import pyplot as plt
import numpy as np
import math

## Cumulative Net Revenue

In [21]:
CumulRevNET = [0]*L
CumulRevNET[0] = RevNET[0]
for k in range (1,L,1):
    CumulRevNET[k] = CumulRevNET[k-1] + RevNET[k]
    
# Export all revenues data to Excel :
    
df2 = pd.DataFrame([RevDA, RevB, RevNET, CumulRevNET])
df2.to_excel("Revenue2.xlsx")
print('Exported')


Exported


## Forecast reliability

In [22]:
ERROR = [0]*L
DeltaLAMBDA = [0]*L                                # Difference between clearing and applied balancing price under 2-price settlement
relERROR = [0]*L                                   # Relative error with the measure
for k in range (0,L,1):
    ERROR[k] = (SURPLUS[k] + LACK[k])*0.001
    DeltaLAMBDA[k] = LambdaREG[k] - LambdaS[k]
    realMeas = RealMEAS[k]*0.001
    if realMeas == 0:
        relERROR[k] = 0
    else:
        relERROR [k] = ERROR[k]/realMeas

relError = (sum(relERROR)/L)
print('The medium relative forecast error is :')
print(relError)


The medium relative forecast error is :
-2.3149717100052514
